In [1]:
import tensorflow
import matplotlib
import seaborn 
import numpy 
import pandas
import sklearn

print(tensorflow.__version__)
print(matplotlib.__version__)
print(seaborn.__version__)
print(numpy.__version__)
print(pandas.__version__)
print(sklearn.__version__)

2025-10-01 14:25:28.243685: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-01 14:25:28.254890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759296328.270415    6493 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759296328.275767    6493 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-01 14:25:28.292365: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

2.18.0
3.10.6
0.13.2
2.2.6
2.3.2
1.7.2


In [1]:
import tensorflow
import matplotlib
import seaborn 
import numpy 
import pandas
import sklearn

from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB

from xgboost import XGBClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
import lightgbm as lgbm

2025-10-01 15:51:55.749144: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-01 15:51:55.816856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759301515.837391   10859 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759301515.844796   10859 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-01 15:51:55.924105: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Vocabulary Size 10000

In [7]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000, test_split=0.2)

In [8]:
print('훈련 샘플의 수: {}'.format(len(x_train)))
print('테스트 샘플의 수: {}'.format(len(x_test)))

훈련 샘플의 수: 8982
테스트 샘플의 수: 2246


In [9]:
num_classes = max(y_train) + 1
print('클래스의 수 : {}'.format(num_classes))

클래스의 수 : 46


In [11]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
index_to_word = { index+3 : word for word, index in word_index.items() }

for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token


### 학습 데이터 디코딩

In [12]:
#학습 데이터 디코딩
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

#테스트 데이터 디코딩
decoded_test = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded_test.append(t)

x_test = decoded_test 
print(len(x_test))

8982
2246


### DTM, TF-IDF 적용

In [13]:
#DTM
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train) 
x_test_dtm = dtmvector.transform(x_test)

#TF-IDF
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
tfidfv_test = tfidf_transformer.transform(x_test_dtm)

### 모델 학습
- LogisticRegression, SVM, NaiveBayes, KNN, DecisionTree, RandomForest , XGBoost, LightGBM

In [23]:
models = {
    'Logistic Regression': LogisticRegression(C=10000, penalty='l2', max_iter=3000),
    'SVM': LinearSVC(C=1000, penalty='l1', max_iter=7000, dual=False),
    'Naive Bayes': MultinomialNB(),
    'KNN': KNeighborsClassifier(n_neighbors=6),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=0),
    'Random Forest': RandomForestClassifier(n_estimators = 5,random_state = 0),
    'XGBoost': XGBClassifier(tree_method = "hist", device = "cuda",random_state=0),
    'LightGBM': LGBMClassifier(device='cuda', random_state=0,verbosity=-1) 
}

results = []

for name, model in models.items():

    model.fit(tfidfv, y_train)
    predictions = model.predict(tfidfv_test)
    
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'F1 Score': f1
    })

results_df = pd.DataFrame(results)
results_df_sorted = results_df.sort_values(by='F1 Score', ascending=False)

results_df_sorted

/home/coco/anaconda3/envs/project_env/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/coco/anaconda3/envs/project_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,Model,Accuracy,F1 Score
0,Logistic Regression,0.808549,0.802277
6,XGBoost,0.797418,0.790745
1,SVM,0.788068,0.784735
3,KNN,0.788958,0.778244
5,Random Forest,0.674087,0.642948
4,Decision Tree,0.620214,0.577640
2,Naive Bayes,0.656723,0.576447
7,LightGBM,0.322796,0.178131


# Vocabulary Size 5000

In [24]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=5000, test_split=0.2)

In [25]:
print('훈련 샘플의 수: {}'.format(len(x_train)))
print('테스트 샘플의 수: {}'.format(len(x_test)))

훈련 샘플의 수: 8982
테스트 샘플의 수: 2246


In [26]:
num_classes = max(y_train) + 1
print('클래스의 수 : {}'.format(num_classes))

클래스의 수 : 46


In [27]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
index_to_word = { index+3 : word for word, index in word_index.items() }

for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token


### 학습 데이터 디코딩

In [28]:
#학습 데이터 디코딩
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

#테스트 데이터 디코딩
decoded_test = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded_test.append(t)

x_test = decoded_test 
print(len(x_test))

8982
2246


### DTM, TF-IDF 적용

In [29]:
#DTM
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train) 
x_test_dtm = dtmvector.transform(x_test)

#TF-IDF
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
tfidfv_test = tfidf_transformer.transform(x_test_dtm)

### 모델 학습
- LogisticRegression, SVM, NaiveBayes, KNN, DecisionTree, RandomForest , XGBoost, LightGBM

In [30]:
models = {
    'Logistic Regression': LogisticRegression(C=10000, penalty='l2', max_iter=3000),
    'SVM': LinearSVC(C=1000, penalty='l1', max_iter=7000, dual=False),
    'Naive Bayes': MultinomialNB(),
    'KNN': KNeighborsClassifier(n_neighbors=6),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=0),
    'Random Forest': RandomForestClassifier(n_estimators = 5,random_state = 0),
    'XGBoost': XGBClassifier(tree_method = "hist", device = "cuda",random_state=0),
    'LightGBM': LGBMClassifier(device='cuda', random_state=0,verbosity=-1) 
}

results = []

for name, model in models.items():

    model.fit(tfidfv, y_train)
    predictions = model.predict(tfidfv_test)
    
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'F1 Score': f1
    })

results_df = pd.DataFrame(results)
results_df_sorted = results_df.sort_values(by='F1 Score', ascending=False)

results_df_sorted

/home/coco/anaconda3/envs/project_env/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/coco/anaconda3/envs/project_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,Model,Accuracy,F1 Score
0,Logistic Regression,0.805877,0.799952
6,XGBoost,0.800089,0.794667
3,KNN,0.789849,0.778443
1,SVM,0.771149,0.766369
5,Random Forest,0.701247,0.677022
2,Naive Bayes,0.673197,0.601250
4,Decision Tree,0.617988,0.572997
7,LightGBM,0.255120,0.171687


# Vocabulary Size ALL

In [31]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

In [32]:
print('훈련 샘플의 수: {}'.format(len(x_train)))
print('테스트 샘플의 수: {}'.format(len(x_test)))

훈련 샘플의 수: 8982
테스트 샘플의 수: 2246


In [33]:
num_classes = max(y_train) + 1
print('클래스의 수 : {}'.format(num_classes))

클래스의 수 : 46


In [34]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
index_to_word = { index+3 : word for word, index in word_index.items() }

for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

### 학습 데이터 디코딩

In [35]:
#학습 데이터 디코딩
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

#테스트 데이터 디코딩
decoded_test = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded_test.append(t)

x_test = decoded_test 
print(len(x_test))

8982
2246


### DTM, TF-IDF 적용

In [36]:
#DTM
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train) 
x_test_dtm = dtmvector.transform(x_test)

#TF-IDF
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
tfidfv_test = tfidf_transformer.transform(x_test_dtm)

### 모델 학습
- LogisticRegression, SVM, NaiveBayes, KNN, DecisionTree, RandomForest , XGBoost, LightGBM

In [37]:
models = {
    'Logistic Regression': LogisticRegression(C=10000, penalty='l2', max_iter=3000),
    'SVM': LinearSVC(C=1000, penalty='l1', max_iter=7000, dual=False),
    'Naive Bayes': MultinomialNB(),
    'KNN': KNeighborsClassifier(n_neighbors=6),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=0),
    'Random Forest': RandomForestClassifier(n_estimators = 5,random_state = 0),
    'XGBoost': XGBClassifier(tree_method = "hist", device = "cuda",random_state=0),
    'LightGBM': LGBMClassifier(device='cuda', random_state=0,verbosity=-1) 
}

results = []

for name, model in models.items():

    model.fit(tfidfv, y_train)
    predictions = model.predict(tfidfv_test)
    
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'F1 Score': f1
    })

results_df = pd.DataFrame(results)
results_df_sorted = results_df.sort_values(by='F1 Score', ascending=False)

results_df_sorted

/home/coco/anaconda3/envs/project_env/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/coco/anaconda3/envs/project_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,Model,Accuracy,F1 Score
0,Logistic Regression,0.811220,0.805548
6,XGBoost,0.791630,0.784818
1,SVM,0.788513,0.784226
3,KNN,0.773820,0.765260
5,Random Forest,0.654497,0.622591
4,Decision Tree,0.621104,0.576928
2,Naive Bayes,0.599733,0.504567
7,LightGBM,0.274711,0.271040


# Vocabulary Size 10000 with DL

In [2]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

word_index = reuters.get_word_index(path="reuters_word_index.json")
index_to_word = { index+3 : word for word, index in word_index.items() }

for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

### 학습 데이터 디코딩

In [3]:
#학습 데이터 디코딩
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

#테스트 데이터 디코딩
decoded_test = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded_test.append(t)

x_test = decoded_test 
print(len(x_test))

8982
2246


### W2V 적용

In [5]:
from gensim.models import Word2Vec
import numpy as np

x_train_tokenized = [sentence.split() for sentence in x_train]
x_test_tokenized = [sentence.split() for sentence in x_test]

model = Word2Vec(sentences = x_train_tokenized, vector_size = 256, window = 5, min_count = 5, workers = 4, sg = 0)
w2v_model = model

def vectorize_sentence(sentence, model, max_len):
    vecs = []
    for word in sentence:
        if word in model.wv:
            vecs.append(model.wv[word])
        else:
            vecs.append(np.zeros(model.vector_size))
    
    if len(vecs) < max_len:
        vecs += [np.zeros(model.vector_size)] * (max_len - len(vecs))
    else:
        vecs = vecs[:max_len]
    return np.array(vecs)

#DL용 데이터
x_train_w2v = np.array([vectorize_sentence(s, w2v_model, max_len=100) for s in x_train_tokenized])
x_test_w2v = np.array([vectorize_sentence(s, w2v_model, max_len=100) for s in x_test_tokenized])

In [6]:
#ML용 데이터
x_w2v_seq_train = x_train_w2v
x_w2v_seq_test = x_test_w2v

x_w2v_avg_train = np.mean(x_w2v_seq_train, axis=1)
x_w2v_avg_test = np.mean(x_w2v_seq_test, axis=1)
print(x_w2v_avg_train.shape)  # (8982, 256)

(8982, 256)


## 모델 학습
### LogisticRegression

In [18]:
# LogisticRegression 모델

lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000)
lr.fit(x_w2v_avg_train, y_train)
predicted = lr.predict(x_w2v_avg_test)

name = 'Logistic Regression'
accuracy = accuracy_score(y_test, predicted)
f1 = f1_score(y_test, predicted, average='weighted')

results = []

results.append({
    'Model': name,
    'Accuracy': accuracy,
    'F1 Score': f1
})

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.7351
F1-score: 0.7343


### DNN

In [19]:
#DNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM, Dense

dense_model = Sequential([
    Flatten(input_shape=(100, 256)),  # (seq_len, embedding_dim)
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(46, activation='softmax')   # 클래스 수에 맞게 조정 46개로 맞춰주세요!
])

dense_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dense_model.summary()

/home/coco/anaconda3/envs/project_copy/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │    13,107,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 46)             │         5,934 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,277,870 (50.65 MB)

 Trainable params: 13,277,870 (50.65 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
dense_model.fit(x_train_w2v, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.5623 - loss: 1.9521 - val_accuracy: 0.6505 - val_loss: 1.5714
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ -5s -21989us/step - accuracy: 0.6586 - loss: 1.4872 - val_accuracy: 0.6761 - val_loss: 1.4394
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7019 - loss: 1.2570 - val_accuracy: 0.7028 - val_loss: 1.3981
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7353 - loss: 1.1175 - val_accuracy: 0.6884 - val_loss: 1.3632
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7576 - loss: 0.9775 - val_accuracy: 0.6895 - val_loss: 1.4188
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7741 - loss: 0.9152 - val_accuracy: 0.6761 - val_loss: 1.5206
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7961 - loss: 0.8383 - val_accuracy: 0.6834 - val_loss: 1.5707
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8104 - loss: 0.7564 - val_accu

In [21]:
y_pred_proba = dense_model.predict(x_test_w2v)
y_pred = np.argmax(y_pred_proba, axis=1)

name = "Dense model"
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

results.append({
    'Model': name,
    'Accuracy': acc,
    'F1 Score': f1
})

print(f"Accuracy: {acc:.4f}")
print(f"F1-score: {f1:.4f}")

71/71 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step  
Accuracy: 0.6825
F1-score: 0.6551


### RNN

In [22]:
#RNN model

rnn_model = Sequential([
    LSTM(128, input_shape=(100, 256)),  # (seq_len, embedding_dim)
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(46, activation='softmax')   # 클래스 수에 맞게 조정 46개로 맞춰주세요~
])

rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
rnn_model.summary()

/home/coco/anaconda3/envs/project_copy/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 46)             │         2,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 208,366 (813.93 KB)

 Trainable params: 208,366 (813.93 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
rnn_model.fit(x_train_w2v, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.4860 - loss: 2.1063 - val_accuracy: 0.5843 - val_loss: 1.6927
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.5894 - loss: 1.6808 - val_accuracy: 0.6177 - val_loss: 1.6358
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.6294 - loss: 1.5526 - val_accuracy: 0.6594 - val_loss: 1.4278
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ -2s -8080us/step - accuracy: 0.6604 - loss: 1.4034 - val_accuracy: 0.7006 - val_loss: 1.2691
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.6849 - loss: 1.3087 - val_accuracy: 0.7067 - val_loss: 1.2377
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.6998 - loss: 1.2360 - val_accuracy: 0.7101 - val_loss: 1.2003
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.7063 - loss: 1.1899 - val_accuracy: 0.7056 - val_loss: 1.1957
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.7169 - loss: 1.1277 - val

In [24]:
y_pred_proba = rnn_model.predict(x_test_w2v)
y_pred = np.argmax(y_pred_proba, axis=1)

name = "RNN model"
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

results.append({
    'Model': name,
    'Accuracy': acc,
    'F1 Score': f1
})

print(f"Accuracy: {acc:.4f}")
print(f"F1-score: {f1:.4f}")

71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Accuracy: 0.7404
F1-score: 0.7120


In [25]:
results_df = pd.DataFrame(results)
results_df_sorted = results_df.sort_values(by='F1 Score', ascending=False)

results_df_sorted

,Model,Accuracy,F1 Score
0,Logistic Regression,0.735085,0.734349
2,RNN model,0.740427,0.711959
1,Dense model,0.682547,0.655123
